In [1]:
data_dir = '/home/zhulei/Data/pg_data/Images'
query_dir = '/home/zhulei/Data/pg_data/Queries'

In [2]:
import os

im_bb_path_tuples = [(os.path.join(query_dir, '{:02d}.jpg'.format(i)),
                      os.path.join(query_dir, '{:02d}.txt'.format(i))
                     ) for i in range(1, 51)
                    ]

In [3]:
from image_db_v2 import ImageDB

cluster_im_num = 2048
vsize = 2048

im_db_copy = ImageDB()
im_db_copy.load('cin2048_vsize2048_db.joblib')
im_db_copy.switch_sim_metric('cosine')
# im_db_copy.migrate_to_tfidf()
# im_db_copy.save('cin{:d}_vsize{:d}_tfidf_db.joblib'.format(cluster_im_num, vsize))

2020-03-22 07:00:17,846 - INFO - Rebuilding KNN for image retrieval...
2020-03-22 07:00:17,848 - INFO - Building Finished!


In [4]:
result_dir = 'result'
os.makedirs(result_dir, exist_ok=True)

In [8]:
import pandas as pd
from tqdm import tqdm
import string
import time

DEBUG = False

def im_path_to_id(im_path):
    im_name = os.path.basename(im_path)
    im_id = int(os.path.splitext(im_name)[0]) # to remove leading zeros
    return str(im_id)

num_imgs = len(os.listdir(data_dir))
print('number of images in database: {:d}'.format(num_imgs))
print('processing queries...')
time.sleep(1) # just to avoid print function to interupt progress bar which makes me uncomfortable

rankfile = os.path.join(result_dir, 'rankList.txt')
with open(rankfile, 'w') as f:
    i = 1
    for (im_path, bb_path) in tqdm(im_bb_path_tuples, disable=DEBUG):
        similar_imgs =  im_db_copy.retrieve_and_locate(im_path, bb_path, top_k=num_imgs, locate=False)
        line = 'Q{:d}: '.format(i)
        if DEBUG:
            print(line+' '.join([ im_path_to_id(x[0]) for x in similar_imgs[:10] ]))
            print('score: ' + ' '.join([ '{:.4f}'.format(x[1]) for x in similar_imgs[:10] ]))
        
        line += ' '.join([ im_path_to_id(x[0]) for x in similar_imgs ])
#         line += ' '.join([ im_path_to_id(v[0]) for k, v in similar_imgs.items() ][1:])
        line += '\n'
        f.writelines(line)
        i += 1
        

number of images in database: 28493
processing queries...


100%|██████████| 50/50 [00:32<00:00,  1.56it/s]
